# 준비

In [0]:
import sys
IN_COLAB = 'google.colab' in sys.modules


## 패키지 설치

In [0]:
!pip install pandas numpy tqdm==4.43.0 bayesian-optimization lightgbm

## 라이브러리 임포트

In [0]:
import sys
import pandas as pd                         # 데이터 분석 라이브러리
# import numpy as np                          # 계산 라이브러리
# from tqdm.auto import tqdm                  # 진행바
# from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
# from sklearn.model_selection import KFold   # K-fold CV    
# from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
# from functools import partial               # 함수 변수 고정
# import lightgbm as lgb                      # LightGBM 라이브러리
# import multiprocessing
# import gc
import importlib
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
# tqdm.pandas()

In [0]:
def reload_module(module):
    for _ in range(2):
        for k, v in sys.modules.items():
            if k.startswith(module.__name__):
                importlib.reload(v)

## 상수 정의

In [0]:
DATA_DIR = 'data'

## Colab 설정

In [0]:
if IN_COLAB:
    import getpass
    import urllib.parse

    github_repo = "github.com/Heo-H/dacon"
    branch_name = "dacon-202003"

    print("Please enter your Github account.")
    github_username = urllib.parse.quote(input("username:"))
    github_password = urllib.parse.quote(getpass.getpass("password:"))
    github_url = f"https://{github_username}:{github_password}@{github_repo}"
    github_sealed_url = f"https://{github_username}:****@{github_repo}"

    output = !git clone -b $branch_name --single-branch $github_url tmp_dacon_202003 && mv tmp_dacon_202003/* . && rm -r tmp_dacon_202003
    for line in output:
        print(line.replace(github_url, github_sealed_url))

    del github_repo
    del github_username
    del github_password
    del github_url
    del github_sealed_url
    del branch_name

In [0]:
if IN_COLAB:
    DRIVE_DIR = '/content/drive'
    from google.colab import drive
    drive.mount(DRIVE_DIR)

    import os
    os.symlink(f'{DRIVE_DIR}/My Drive/data', DATA_DIR)

# 학습 데이터

In [0]:
train = pd.read_csv('data/train.csv')

# 특징 추출

In [0]:
import sclearn.feature_engineering
reload_module(sclearn.feature_engineering)

x_train = sclearn.feature_engineering.prepare_x_data(train)
y_train = sclearn.feature_engineering.prepare_y_data(train)

In [0]:
x_train.head()

# 모델 구축

In [0]:
import sclearn.learning
reload_module(sclearn.learning)

models, lgb_bo, swap_table = sclearn.learning.train_models(x_train, y_train)

# 테스트

In [0]:
test = pd.read_csv('data/test.csv')

In [0]:
import sclearn.feature_engineering
reload_module(sclearn.feature_engineering)

x_test = sclearn.feature_engineering.prepare_x_data(test)

In [0]:
x_test.head()

In [0]:
import sclearn.learning
reload_module(sclearn.learning)

submission = sclearn.learning.test_models(models, x_test)
submission.to_csv('submission.csv')
submission.head()